In [1]:
from clipper_admin import ClipperConnection, DockerContainerManager
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.start_clipper()

20-03-09:03:28:27 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-03-09:03:28:31 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmpx53c2__4.yml
20-03-09:03:28:32 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


In [2]:
#插入排序
def insert_sort(lists):
#     list = lists[0]
    list1 = []
    for l in range(len(lists)):    
        list = lists[l]
        length = len(list)
        for i in range(1,length):
            for j in range(i):
                if list[j] > list[j+1]:
                    list[j],list[j+1] = list[j+1],list[j]
        list1.append(str(list))
    return list1

In [3]:
from clipper_admin.deployers import python as python_deployer

In [4]:
python_deployer.deploy_python_closure(clipper_conn, name="insert-model", version=1, input_type="int", func=insert_sort)

20-03-09:03:29:46 INFO     [deployer_utils.py:41] Saving function to /tmp/tmp30htnxzsclipper
20-03-09:03:29:46 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-03-09:03:29:46 INFO     [python.py:192] Python closure saved
20-03-09:03:29:46 INFO     [python.py:206] Using Python 3.6 base image
20-03-09:03:29:46 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmp30htnxzsclipper
20-03-09:03:29:47 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/python36-closure-container:0.4.1
20-03-09:03:29:47 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
20-03-09:03:29:47 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmp30htnxzsclipper /model/
20-03-09:03:29:47 INFO     [clipper_admin.py:539] [default-cluster]  ---> 8bc597d4bf33
20-03-09:03:29:47 INFO     [clipper_admin.py:539] [default-cluster] Successfully built 8bc597d4bf33
20-03-09:03:29:47 INFO   

In [5]:
clipper_conn.register_application(name="hello-world", input_type="int", default_output="-1.0", slo_micros=100000)

20-03-09:03:33:55 INFO     [clipper_admin.py:236] [default-cluster] Application hello-world was successfully registered


In [6]:
clipper_conn.link_model_to_app(app_name="hello-world", model_name="insert-model")

20-03-09:03:35:42 INFO     [clipper_admin.py:303] [default-cluster] Model insert-model is now linked to application hello-world


In [7]:
clipper_conn.get_all_apps()

['hello-world']

In [8]:
def predict(addr, x, batch=False):
    url = "http://%s/hello-world/predict" % addr

    if batch:
        req_json = json.dumps({'input_batch': x})
    else:
        req_json = json.dumps({'input': list(x)})

    headers = {'Content-type': 'application/json'}
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    print("'%s', %f ms" % (r.text, latency))

In [9]:
import json ,requests
from datetime import datetime
addr = "127.0.0.1:1337"
# x = '{"input": [7,2,3]}'
x = [1,8,2]
predict(addr,x,batch=False)

'{"query_id":0,"output":"[1 2 8]","default":false}', 11.275000 ms


In [10]:
clipper_conn.stop_all()

20-03-09:03:56:13 INFO     [clipper_admin.py:1424] [default-cluster] Stopped all Clipper cluster and all model containers
